## Setup

In [1]:
import sys
sys.path.append("/root/specialised-SAEs")
from datasets import load_dataset
from transformer_lens import utils, HookedTransformer
import gc
import torch
from sae_lens.jacob.load_sae_from_hf import load_sae_from_hf
from config import DTYPE_MAP
from tqdm import tqdm

In [6]:
torch.set_grad_enabled(False)
DTYPE = "bfloat16"
ctx_length = 128
# model = HookedTransformer.from_pretrained_no_processing("gemma-2b-it", device="cuda", dtype=DTYPE)
model = HookedTransformer.from_pretrained_no_processing("gpt2-small", device="cuda", dtype=DTYPE)



config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-small into HookedTransformer


In [7]:
# gsae = load_sae_from_hf("jacobcd52/gemma2-gsae", 
#                         "sae_weights.safetensors", 
#                         "cfg.json",
#                         device="cuda",
#                         dtype=DTYPE)

# ssae_list = [
#     load_sae_from_hf("jacobcd52/gemma2-ssae-phys",
#                     f"l1_coeff={l1_coeff}_tokens=40960000_lr=0.001.safetensors",
#                     f"l1_coeff={l1_coeff}_tokens=40960000_lr=0.001_cfg.json",
#                     device="cuda",
#                     dtype=DTYPE)
#     for l1_coeff in [5, 10, 20]
# ]

sae = load_sae_from_hf("jacobcd52/gpt2-gsae-phys",
                       "l1_coeff=6_expansion=0.5.safetensors",
                          "l1_coeff=6_expansion=0.5_cfg.json",
                          device="cuda",
                          dtype=DTYPE)

# gsae_0_list = [
#     load_sae_from_hf("jacobcd52/gemma2-gsae-finetune-phys",
#                     f"l1_coeff={l1_coeff}_tokens=10240000_lr=0.0001.safetensors",
#                     f"l1_coeff={l1_coeff}_tokens=10240000_lr=0.0001_cfg.json",
#                     device="cuda",
#                     dtype=DTYPE)
#     for l1_coeff in [1, 2, 5]
# ]

# gsae_0p1_list = [
#     load_sae_from_hf("jacobcd52/gemma2-gsae-finetune-phys",
#                     f"l1_coeff={l1_coeff}_tokens=10240000_lr=0.0001_control_mix=0.1.safetensors",
#                     f"l1_coeff={l1_coeff}_tokens=10240000_lr=0.0001_control_mix=0.1_cfg.json",
#                     device="cuda",
#                     dtype=DTYPE)
#     for l1_coeff in [1, 2, 5]
# ]

# gsae_0p5_list = [
#     load_sae_from_hf("jacobcd52/gemma2-gsae-finetune-phys",
#                     f"l1_coeff={l1_coeff}_tokens=10240000_lr=0.0001_control_mix=0.5.safetensors",
#                     f"l1_coeff={l1_coeff}_tokens=10240000_lr=0.0001_control_mix=0.5_cfg.json",
#                     device="cuda",
#                     dtype=DTYPE)
#     for l1_coeff in [1, 2, 5]
# ]

l1_coeff=6_expansion=0.5.safetensors:   0%|          | 0.00/2.37M [00:00<?, ?B/s]

GSAE weights file saved as temp_sae/sae_weights.safetensors


l1_coeff=6_expansion=0.5_cfg.json:   0%|          | 0.00/2.44k [00:00<?, ?B/s]

GSAE cfg file saved as temp_sae/cfg.json
Loading weights into GSAE from temp_sae/sae_weights.safetensors
temp_sae/cfg.json temp_sae/sae_weights.safetensors


In [8]:
# get OWT tokens
data = load_dataset("stas/openwebtext-10k", split="train")
tokenized_data = utils.tokenize_and_concatenate(data, model.tokenizer, max_length=ctx_length)
tokenized_data = tokenized_data.shuffle(42)
owt_tokens = tokenized_data["tokens"][:20_000].cuda()
print("owt_tokens has shape", owt_tokens.shape)
print("total number of tokens:", int(owt_tokens.numel()//1e6), "million")
print()

# get physics-papers tokens
data = load_dataset("jacobcd52/physics-papers", split="train[:10%]")
# Define a filter function to remove null entries
def remove_null_entries(example):
    return all(value is not None and value != '' for value in example.values())
# Apply the filter to remove null entries
data = data.filter(remove_null_entries)
tokenized_data = utils.tokenize_and_concatenate(data, model.tokenizer, max_length=ctx_length)
tokenized_data = tokenized_data.shuffle(42)
phys_tokens = tokenized_data["tokens"][:20_000].cuda()
print("phys_tokens has shape", phys_tokens.shape)
print("total number of tokens:", int(phys_tokens.numel()//1e6), "million")

# clean up
del tokenized_data, data
gc.collect()

Map (num_proc=10):   0%|          | 0/10000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (56043 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (55582 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (56550 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (59222 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (58623 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence

owt_tokens has shape torch.Size([20000, 128])
total number of tokens: 2 million



Map (num_proc=10):   0%|          | 0/1570 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (61306 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (78162 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (106471 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (114075 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (130021 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum seque

phys_tokens has shape torch.Size([20000, 128])
total number of tokens: 2 million


0

## Frequency Histograms

In [9]:
import gc
from einops import rearrange
def get_freqs_and_mean_acts(sae_list, tokens, num_tokens=10_000, batch_size=16):
    hook_pt = sae_list[0].cfg.hook_name
    layer = sae_list[0].cfg.hook_layer
    num_batches = num_tokens // (ctx_length*batch_size)
    num_tokens = num_batches * (ctx_length-1) * batch_size

    counts = {i : torch.zeros(sae.cfg.d_sae, dtype=torch.int32).cuda() 
              for i, sae in enumerate(sae_list)}

    for b in tqdm(range(num_batches)):
        batch = tokens[b*batch_size:(b+1)*batch_size]
        loss, cache = model.run_with_cache(
            batch,
            return_type="loss",
            names_filter=[hook_pt],
            prepend_bos=False
        )
        flat_acts = rearrange(cache[hook_pt][:, 1:, :], "b s d -> (b s) d")
        for i, sae in enumerate(sae_list):
            is_active = (sae.encode_fn(flat_acts) > 0).to(torch.int32)
            counts[i] += is_active.sum(dim=0)

        del cache
        gc.collect()
    freqs = [count.cpu().detach() / num_tokens for count in counts.values()]

    return freqs

In [27]:
# all_sae_list = [gsae] + gsae_0p1_list + ssae_list
phys_freqs = get_freqs_and_mean_acts([sae], phys_tokens, num_tokens=1_000_000, batch_size=128)

  0%|          | 0/61 [00:00<?, ?it/s]

100%|██████████| 61/61 [00:26<00:00,  2.29it/s]


In [29]:
owt_freqs = phys_freqs

In [30]:
import plotly.express as px
import pandas as pd

# Create a pandas DataFrame directly from tensors
df = pd.DataFrame({
    'value': (torch.cat([owt_freqs[0], owt_freqs[-1]]) + 1e-8).log10().numpy(),
    'source': ['GSAE'] * len(owt_freqs[0]) + ['SSAE'] * len(owt_freqs[-1])
})

# Create histogram with normalization to probability
fig = px.histogram(df, x='value', color='source', barmode='overlay', 
                   histnorm='probability',
                   title='Phys feature frequencies',
                   labels={'value': 'Value', 'source': 'Source'},
                   nbins=100)

# Show plot
fig.show()

/usr/local/lib/python3.10/dist-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [36]:
px.histogram((torch.cat([owt_freqs[0], owt_freqs[-1]]) +1e-8).log10())

## Physics features in GSAE-finetunes

In [28]:
owt_freqs = get_freqs_and_mean_acts([ssae_list[2]], owt_tokens, num_tokens=1000_000, batch_size=128)
phys_freqs = get_freqs_and_mean_acts([ssae_list[2]], phys_tokens, num_tokens=1000_000, batch_size=128)

100%|██████████| 61/61 [01:40<00:00,  1.65s/it]


In [29]:
import plotly.express as px
px.histogram((owt_freqs[-1] + 1e-8).log10(), title="SSAE feature frequencies: OWT").show()
px.histogram((phys_freqs[-1] + 1e-8).log10(), title="SSAE feature frequencies: phys").show()

In [27]:
ratios = [f_phys / f_owt for f_phys, f_owt in zip(phys_freqs[0], owt_freqs[0]) if f_owt > 0]
px.histogram(ratios)

In [19]:
ids = [i for i in range(len(owt_freqs[0])) 
       if phys_freqs[0][i] > 1e-5
       and phys_freqs[0][i] / owt_freqs[0][i] > 100
       ]
print(len(ids))
print(ids)
print(phys_freqs[0][ids] / owt_freqs[0][ids])

35
[749, 1274, 2346, 2353, 4585, 6447, 8114, 9646, 11407, 11540, 12451, 14503, 14684, 15171, 15246, 15314, 15344, 15536, 16238, 16261, 16545, 16570, 20752, 20916, 22401, 23048, 26239, 29236, 30120, 31045, 31198, 31205, 31530, 31834, 32261]
tensor([144.4737, 285.3333, 223.7778, 113.9143, 221.7333, 109.1383, 142.2857,
        665.0000, 128.0696, 221.5512, 116.3581, 127.6931,      inf, 104.7141,
        164.0000, 221.3424, 136.9951,      inf, 107.4286, 168.3750, 120.5747,
        304.0000, 122.8554, 112.3092, 125.8333, 267.0286,      inf, 207.2500,
        128.4548, 199.3561, 280.5454, 321.2500,      inf, 111.3101, 667.8334])
